In [1]:
import pypdf
import re

fp = "data/sub/O6_1A_Umweltgerechte_und_biodiversitaetsfoerdernde_Bewirtschaftung_(UBB)_2023_04.pdf"

# Create a PDF object
reader = pypdf.PdfReader(fp)

reader.pages[0].extract_text()

'Informationsblatt ÖPUL 2023  \nUmweltgerechte und biodiversitätsfördernde \nBewirtschaftung    Seite 1 von 38 www.eama.at  | www.ama.at   \n  \nÖPUL 2023  \nUmweltgerechte und biodiversitätsfördernde Bewirtschaftung  \nSTAND April 2023 \n \nINHALT  \nTest \n1 Übersicht  ....................................................................................................................... 2 \n2 Zielsetzung  .................................................................................................................... 2 \n3 Teilnahmevoraussetzungen  .......................................................................................... 3 \n3.1 Vertragszeitraum  ................................................................................................... 3 \n3.2 Teilnahmefähige Flächen  ...................................................................................... 3 \n4 Eigenschaft als tierhaltender Betrieb  ..........................................................

In [2]:
# read in text file first_doc_text.txt to further process
with open('first_doc_text.txt') as f:
    txt = f.read()

txt

'Informationsblatt ÖPUL 2023  \nUmweltgerechte und biodiversitätsfördernde \nBewirtschaftung    Seite 1 von 38 www.eama.at  | www.ama.at   \n  \nÖPUL 2023  \nUmweltgerechte und biodiversitätsfördernde Bewirtschaftung  \nSTAND April 2023 \n \nINHALT  \nTest \n1 Übersicht  ....................................................................................................................... 2 \n2 Zielsetzung  .................................................................................................................... 2 \n3 Teilnahmevoraussetzungen  .......................................................................................... 3 \n3.1 Vertragszeitraum  ................................................................................................... 3 \n3.2 Teilnahmefähige Flächen  ...................................................................................... 3 \n4 Eigenschaft als tierhaltender Betrieb  ..........................................................

In [3]:
# match (?<=ÖPUL 2023\s*).*?(?=\s*STAND)
matches = re.findall(r'ÖPUL 2023\s*([\s\S]*?)\s*STAND', txt, re.DOTALL)
matches


['Umweltgerechte und biodiversitätsfördernde \nBewirtschaftung    Seite 1 von 38 www.eama.at  | www.ama.at   \n  \nÖPUL 2023  \nUmweltgerechte und biodiversitätsfördernde Bewirtschaftung']

In [4]:
import pdfplumber

# Code to extract chapters and subchapters from pdf
def extract_chapters(pdf_path):
    data = {'chapters': []}

    with pdfplumber.open(pdf_path) as pdf:
        pages = range(len(pdf.pages))
        for i in pages:
            page = pdf.pages[i]
            text = page.extract_text()
            lines = text.split('\n')
            current_chapter = None
            current_subchapter = None

            for line in lines:
                if line.isupper():  # Identifying chapters
                    current_chapter = line.strip()
                    data['chapters'].append(
                        {'title': current_chapter, 'subchapters': []})
                elif current_chapter and line.strip() and not line.isnumeric():  # Identifying subchapters
                    current_subchapter = line.strip()
                    data['chapters'][-1]['subchapters'].append(
                        {'title': current_subchapter, 'text': []})
                elif current_chapter and current_subchapter and line.strip():  # Adding text to subchapter
                    data['chapters'][-1]['subchapters'][-1]['text'].append(
                        line.strip())

    return data


structured_pdf = extract_chapters(fp)

# dump as json with encoding utf-8
import json
with open('structure.json', 'w', encoding='utf-8') as f:
    json.dump(structured_pdf, f, ensure_ascii=False, indent=4)


In [5]:
def extract_title(structured_pdf):
    title = [chapter for chapter in structured_pdf['chapters']
             if chapter['title'] == 'ÖPUL 2023'][0]['subchapters'][0]['title']

    return title


extract_title(structured_pdf)


'Umweltgerechte und biodiversitätsfördernde Bewirtschaftung'

In [10]:
import json
def extract_chapters(structured_pdf):
    content_list_raw  = title = [chapter for chapter in structured_pdf['chapters']
                       if chapter['title'] == 'INHALT'][0]['subchapters'][1:]
    
    content_list = [item['title'] for item in content_list_raw]

    cleaned_data = [re.sub(
        r'(\b\w+\b)\s+\.{3,}\s+(\d+)$', r'\1 \2', item) for item in content_list]
    
    hierarchical_data = {}

    for item in cleaned_data:
        line = item.rstrip('.')
        parts = line.split(' ', 1)
        number, title = parts[0], parts[1]
        number_parts = number.split('.')
        current_dict = hierarchical_data
        for num in number_parts:
            if num not in current_dict:
                current_dict[num] = {}
            current_dict = current_dict[num]

        stripped_title = title.strip()
        current_dict['title'] = " ".join(stripped_title.split(" ")[0:-1])
        current_dict['page'] = stripped_title.split(" ")[-1]

    return hierarchical_data


hierarchical_data = extract_chapters(structured_pdf)

with open('hierarchy.json', 'w', encoding='utf-8') as f:
    json.dump(hierarchical_data, f, ensure_ascii=False, indent=4)


In [24]:
# link some chapter to structure

chapter = structured_pdf['chapters'][20]

index = chapter['title'].split(' ')[0]
title = chapter['title'].split(' ')[1:]
text = " ".join([line['title'] for line in chapter['subchapters']])

# find index and get structure
print(f'Index: {index}')
print(f'Title: {title}')
text




Index: 6.1.4.2
Title: ['PFLEGE-/NUTZUNGSAUFLAGEN']


'Die Biodiversitätsfläche muss mindestens 1x jedes zweite Jahr gemäht oder gehäckselt werden. Mähen/Häckseln ist maximal 2x pro Jahr erlaubt. Das Mähgut darf von der Fläche verbracht und genutzt werden. Auf 75 % der Biodiversitätsflächen ist je Kalenderjahr Mähen/Häckseln frühestens am 1. August erlaubt, auf den anderen 25 % ist dies ohne zeitliche Einschränkungen möglich. Für die Berechnung der DIV-Fläche, welche frühestens am 1. August (75 %) gehäckselt/gemäht werden darf, müssen alle DIV-Flächen unabhängig von der möglichen zusätzlichen Codierung (BAW, AG, NAT, EBW) berücksichtigt werden. Beispiel: Ein Betrieb mit 2,5 ha Grünbrache + DIV und 1,5 ha Grünbrache + DIV + NAT (Projektbestätigung Häckseln ab 01.07.): Gesamte DIV-Fläche: 4 ha, davon 75 % = 3 ha frühestens am 1. August, 1 ha vor dem 1. August (25 %) Folgendes ist erlaubt: • 1,5 ha Grünbrache + DIV+ NAT: Häckseln ab 1. Juli gemäß Projektbestätigung • 2,5 ha Grünbrache + DIV: Häckseln/Mähen ab 1. August Nachdem die Projektbes

In [57]:
doc__title = extract_title(structured_pdf)

header_stack = [doc__title]
prev_index = None



chapters = structured_pdf['chapters'][3:20]
for chapter in chapters:
    index = chapter['title'].split(' ')[0]
    index_list = index.split('.')

    title = " ".join(chapter['title'].split(' ')[1:]).strip()
    text = " ".join([line['title'] for line in chapter['subchapters']])

    header_stack_len = len(header_stack) - 1
    index_len = len(index_list)

    # If new subheader present add it
    if header_stack_len < index_len:
        header_stack.append(title)
    # If going back in hierarchy remove elements to be shorter than index
    elif header_stack_len > index_len:
        [header_stack.pop() for _ in range(header_stack_len - index_len + 1)]
        header_stack.append(title)
    else:
        header_stack.pop()
        header_stack.append(title)
    
    print(f'Index: {index}')
    print(f'Title: {title}')
    print(f'Header Stack: {header_stack}\n')

    


Index: 1
Title: ÜBERSICHT
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Bewirtschaftung', 'ÜBERSICHT']

Index: 2
Title: ZIELSETZUNG
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Bewirtschaftung', 'ZIELSETZUNG']

Index: 3
Title: TEILNAHMEVORAUSSETZUNGEN
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Bewirtschaftung', 'TEILNAHMEVORAUSSETZUNGEN']

Index: 3.1
Title: VERTRAGSZEITRAUM
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Bewirtschaftung', 'TEILNAHMEVORAUSSETZUNGEN', 'VERTRAGSZEITRAUM']

Index: 3.2
Title: TEILNAHMEFÄHIGE FLÄCHEN
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Bewirtschaftung', 'TEILNAHMEVORAUSSETZUNGEN', 'TEILNAHMEFÄHIGE FLÄCHEN']

Index: 4
Title: EIGENSCHAFT ALS TIERHALTENDER BETRIEB
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Bewirtschaftung', 'EIGENSCHAFT ALS TIERHALTENDER BETRIEB']

Index: 5
Title: ALLGEMEINE FÖRDERBEDINGUNGEN
Header Stack: ['Umweltgerechte und biodiversitätsfördernde Be

In [53]:
["x" for i in range(1)]

['x']